In [1]:
import os
from pathlib import Path
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Building Sensitivity Analisys


The aim of this tutorial is to provide a complete workflow for building thermal simulation sensitivity analysis using
[__EnergyPlus__](https://energyplus.net) and __energytoot__

## Introduction

Sensitivity Analysis methods are mathematical method that quantify the impact of and uncertain parameters on a specific metrics.
Various methods exists such as Morris or Sobol.

During building conception workflow, sensitivity analysis can have various benefits:
- Screen out a number of irrelevant conception variables to focus on the important ones (does the solar absorption of the partition glass-wool have a relevant impact on building heat needs ?)
- Sort  relevant uncertain parameter by influence on the observed metrics.
- Help you quantify the relative importance of the modeled physical phenomenons

In this example, the use case is an old building retrofitting. The objective is to insulate the south facade using double skin.

|               Figure 1: Building picture               |            Figure 2: Building thermal model            |
|:------------------------------------------------------:|:------------------------------------------------------:|
| <img src="resources/building_photo.png"  height="300"> | <img src="resources/building_model.png"  height="300"> |

The designer wants to know the impact of the following variables on the building **heat needs** and on the **thermal comfort**:
- Double skin glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope insulation thickness
- Air infiltration coefficient Q4Pa [m<sup>3</sup>/h.m² @4Pa]



To answer these questions we will use an EnergyPlus building model and energytool class to perform Morris & Sobol sensitivity analysis

## Building modeling

In energytool, the <code>Building</code> class is used to simulate HVAC systems through pre-process and post process methods.
The <code>Building</code> holds and idf file. The user specify hvac system using the <code>system</code> module.

In [2]:
from energytool.building import Building

The path of the idd file of EnergyPlus must be given to the <code>Building</code> class.
Be careful idf file E+ version and idd file version must match

In [3]:
Building.set_idd(Path(r'C:\EnergyPlusV9-4-0'))

Now we instantiate a building with an idf file representing the building thermal model.
The idf can be generated manually or using a software (Openstudio, DesignBuilder).
Keep in mind that the main advantage of the energytool <code>Building</code> class, is to simplify hvac system modeling using pre-process and post-process methods.
Thus, we recommend using <code>IdealLoadAirSystem</code> to model HVAC and Domestic Hot Water production (DHW).

In [4]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_as.idf")

The <code>infos()</code> method display information on the building object

In [5]:
building.infos()

==Building==

Number of occupants : 80.21
Building surface : 750.0895999999999 m²
Building volume : 2299.8318 m3
Zone number : 5

==HVAC systems==

Heating systems : []
Cooling systems : []
Ventilation system : []
Artificial lighting system : []
DHW production : []
PV production : []
Others : []


It is time to specify the building hvac equipments.
Let's use the one present in the <code>system</code> module.
Note that you can use custom class as long as they contain a <code>pre_process()</code> and a <code>post_process()</code> methods

In this example, we will only use a boiler with a cop of 1 as we want to work with building heating needs.
For more information on building system, see the dedicated tutorial.

In [6]:
import energytool.system as st

In [7]:
building.heating_system["Main_heater"] = st.HeaterSimple(
    name="IdealBoiler",
    building=building,
    zones='*',
    cop=1
)

We specify the parameters we are uncertain about using the class <code>UncertainParameter</code> from the <code>energytool.parameter</code> module.

In [8]:
from energytool.parameter import UncertainParameter

A <code>list</code> of <code>UncertainParameter</code> is created.
The code below details the configuration for the first object. It corresponds to the SHGC of the external windows.
We specify a relative uncertainty of +- 10% of the idf value:

<pre><code>
UncertainParameter(
    name="SHGC_ext_windows",
    absolute=False,
    bounds=[0.9, 1.1],
    building=building,
    idf_parameters=[dict(
        idf_object="WindowMaterial:SimpleGlazingSystem",
        names='Simple DSF_ext_south_glazing - 1002',
        field='Solar_Heat_Gain_Coefficient',
    )]
</code></pre>

- <code>absolute</code>: if set to <code>True</code> the <code>bounds</code> holds the true values of the parameter. If set to <code>False</code> specify relative <code>bounds</code>. The nominal value is the one set in the idf.
- <code>bounds</code>: the minimum and maximum value of the uncertain parameter.
- <code>idf_parameters</code>: a python the <code>dict</code> with the following keys:
    - <code>idf_object</code> idf object category
    - <code>names</code> a specific idf object name. a list of idf objects names. Default '*' indicates it applies to all specified <code>idf_object</code>
    - <code>field</code> uncertain numerical field


In [9]:
uncertain_param_list = [
    UncertainParameter(
        name="SHGC_ext_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_ext_south_glazing - 1002',
            field='Solar_Heat_Gain_Coefficient',
        )]
    ),
    UncertainParameter(
        name="UFactor_ext_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_ext_south_glazing - 1002',
            field='UFactor',
        )]
    ),
    UncertainParameter(
        name="SHGC_int_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_int_south_glazing - 1001',
            field='Solar_Heat_Gain_Coefficient',
        )]
    ),
    UncertainParameter(
        name="UFactor_int_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_int_south_glazing - 1001',
            field='UFactor',
        )]
    ),
    UncertainParameter(
        name="Wall_insulation_thickness",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="Material",
            names='Wall_insulation_.1',
            field='Thickness',
        )]
    ),
    UncertainParameter(
        name="Cracks",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="AirflowNetwork:MultiZone:Surface:Crack",
            names='*',
            field='Air_Mass_Flow_Coefficient_at_Reference_Conditions',
        )]
    ),
]

Import the sensitivity analysis class <code>SAnalysis</code>
As a minimal configuration, it requires the <code>Building</code> instance, en sensitivity analysis method and the previously defined list of uncertain parameter.

In [25]:
from energytool.sensitivity import SAnalysis

## Morris method
To screen out parameters or to have a first estimation of the uncertain parameters rank without running too many simulation, it is often a good idea to us the Morris method.

In [11]:
sa_analysis = SAnalysis(
    building=building,
    sensitivity_method="Morris",
    parameters=uncertain_param_list,
)

The <code>draw_sample</code> method of <code>SAnalysis</code> draws parameters values according to the <code>parameters</code> list. The sampling method depends on the <code>sensitivity_method</code>. For Morris a One At a Time method is used (OAT). See [SALib documentation](https://salib.readthedocs.io/en/latest/index.html) for more information

The number of trajectories is set to 15.

In [13]:
sa_analysis.draw_sample(n=15)

Sampling results are stored in <code>sample</code>. Columns corresponds to parameters.
Index lines corresponds to a configuration (combination of parameters values)

In [15]:
sa_analysis.sample

array([[0.96666667, 1.03333333, 0.9       , 0.96666667, 0.9       ,
        0.96666667],
       [0.96666667, 1.03333333, 0.9       , 1.1       , 0.9       ,
        0.96666667],
       [1.1       , 1.03333333, 0.9       , 1.1       , 0.9       ,
        0.96666667],
       [1.1       , 1.03333333, 0.9       , 1.1       , 1.03333333,
        0.96666667],
       [1.1       , 1.03333333, 1.03333333, 1.1       , 1.03333333,
        0.96666667],
       [1.1       , 1.03333333, 1.03333333, 1.1       , 1.03333333,
        1.1       ],
       [1.1       , 0.9       , 1.03333333, 1.1       , 1.03333333,
        1.1       ],
       [1.1       , 0.9       , 1.03333333, 1.03333333, 0.96666667,
        1.03333333],
       [1.1       , 1.03333333, 1.03333333, 1.03333333, 0.96666667,
        1.03333333],
       [0.96666667, 1.03333333, 1.03333333, 1.03333333, 0.96666667,
        1.03333333],
       [0.96666667, 1.03333333, 1.03333333, 0.9       , 0.96666667,
        1.03333333],
       [0.96666667, 1

<code>run_simulations</code> method runs the 105 simulations

In [16]:
sa_analysis.run_simulations(epw_file_path=Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw")

<code>analyse</code> method compute sensitivity index depending on the specified <code>sensitivity_method</code> for the specified <code>indicator</code>. By default it will compute sensitivity index for the <code>'Total'</code> hvac energy consumption.

Obtain available indicators using the property <code>available_indicators</code>


In [18]:
sa_analysis.available_indicators

['IdealBoiler_Energy', 'Total']

In [19]:
sa_analysis.analyze(indicator='IdealBoiler_Energy')

Sensitivity index results are stored in <code>sensitivity_results</code>.
Pre-formatted figure for Morris results is available using <code>plot_morris_scatter</code>

In [22]:
from energytool.sensitivity import plot_morris_scatter

In [24]:
plot_morris_scatter(salib_res=sa_analysis.sensitivity_results, title='Building heat needs', unit='J', autosize=True)

In the figure above:
- Circle size indicate the total effect of the parameter on the chosen indicator. The bigger, the more influential.
- The x axis is the mean elementary effect of the parameters. It represents "linear" effect of the parameter.
- The y axis is the standard deviation. It represents interactions between parameters and non linearities.
- The 3 lines separates the figure in 4 regions. From the closer to the x axis : linear, monotonic, almost monotonic and non-linear and/or non-monotonic effects. See [publication](http://www.ibpsa.org/proceedings/BSO2016/p1101.pdf) for more details
- The segment represent the uncertainty on the sensitivity index calculation.

In this use case. Several conclusions can be drawn:
- 4 parameters have an influence on the chosen indicator. Two indicators can be neglected.
- The 4 main parameters have an almost linear influence on the indicator
- The confidence on the sensitivity index calculation is high

## Sobol method

Sobol index indicates the contribution of each uncertain parameters to the variance of the output indicator.
It is a more accurate method to quantify the effect of an uncertain parameter. The second order index also gives more information on the effect of parameters interactions.
... But it comes at a much higher computational cost.
In energytool, the index are computed using SALib. The method gives an estimation of the index value. It reduces the simulation sample size.

Below is an example of a <code>SAnalisys</code> configuration to perform Sobol index calculation
It is very similar to Morris

In [26]:
sa_analysis_sob = SAnalysis(
    building=building,
    sensitivity_method="Sobol",
    parameters=uncertain_param_list,
)

In [28]:
# Salib command an n = 2^x. In this case x shall be >= 6
sa_analysis_sob.draw_sample(n=2**6)

In [29]:
sa_analysis_sob.run_simulations(epw_file_path=Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw")

In [34]:
sa_analysis_sob.analyze(indicator='IdealBoiler_Energy')

Similarly to Morris, a function is designed to plot preformatted Sobol total index graph

In [38]:
from energytool.sensitivity import plot_sobol_st_bar

In [39]:
plot_sobol_st_bar(sa_analysis_sob.sensitivity_results)

The Sobol total index represent an uncertain parameter single effect plus the sum of all its interactions on the considered indicator.
The sum of all the index shall be equal to one.
Salib computes an estimation of this index. Therefore, the sum of the above index is ~0.94. The uncertainty bar shows the confidence interval of the index value.

In this use case, the Sobol method sorted the uncertain parameters in the same order as Morris.
However, the confidence intervals of the index overlap and the sum is around 0.94. A bigger sample could improve the results.

## TODO Second order Sobol index